In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.3 MB/s eta 0:00:00


In [2]:
!pwd
!ls -aF
#!mv optuna.db drive/MyDrive/GitHub/optuna-book/chapter2/

/content
./  ../  .config/  drive/  sample_data/


In [3]:
%cd drive/MyDrive/GitHub/optuna-book/chapter2/
!ls -aF

/content/drive/MyDrive/GitHub/optuna-book/chapter2
adult/							   list_2_19_load_study.py
adult.zip						   list_2_20_and_later_visualization.ipynb
ch2-exec.ipynb						   list_2_4_optimize_beale.py
list_2_12_rf.py						   logs/
list_2_14_optimize_rf.py				   optuna.db
list_2_16_optimize_rf_gb_with_conditional_search_space.py  optuna-prepared.db


In [ ]:
import random
import optuna
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# データのダウンロードと前処理

In [ ]:
# データのダウンロードと前処理
# fetch_openml()で2つ警告が出ている
# 1つ目の警告は、adultデータセットには複数のバージョンがあり、version1が使われた
# 2つ目の警告は、fetch_openml()のparserという引数のデフォルト値がsklearnの1.4から変わった
data = fetch_openml(name="adult")
X = pd.get_dummies(data["data"])
y = [1 if d == ">50K" else 0 for d in data["target"]]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name adult exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
print( type(data["data"]), data["data"].shape )
print( X.shape ) # age: 5, workclass: 8, education: 16, marital-status: 7, occupation: 14, relationship: 6, race: 5, sex: 2, capitalgain: 5, capitalloss: 5, hoursperweek: 5, native-country: 41 -> 合計119、数値が2なので計121
print( X.columns )

<class 'pandas.core.frame.DataFrame'> (48842, 14)
(48842, 121)
Index(['fnlwgt', 'education-num', 'age_0', 'age_1', 'age_2', 'age_3', 'age_4',
       'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=121)


In [ ]:
data["data"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  category
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  float64 
 3   education       48842 non-null  category
 4   education-num   48842 non-null  float64 
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capitalgain     48842 non-null  category
 11  capitalloss     48842 non-null  category
 12  hoursperweek    48842 non-null  category
 13  native-country  47985 non-null  category
dtypes: category(12), float64(2)
memory usage: 1.3 MB


In [ ]:
pd.set_option( 'display.max_rows', 150 )
pd.set_option( 'display.max_columns', 150 )
print( X.head() )
pd.reset_option( 'display.max_rows' )
pd.reset_option( 'display.max_columns' )

     fnlwgt  education-num  age_0  age_1  age_2  age_3  age_4  \
0   77516.0           13.0      0      0      1      0      0   
1   83311.0           13.0      0      0      0      1      0   
2  215646.0            9.0      0      0      1      0      0   
3  234721.0            7.0      0      0      0      1      0   
4  338409.0           13.0      0      1      0      0      0   

   workclass_Federal-gov  workclass_Local-gov  workclass_Never-worked  \
0                      0                    0                       0   
1                      0                    0                       0   
2                      0                    0                       0   
3                      0                    0                       0   
4                      0                    0                       0   

   workclass_Private  workclass_Self-emp-inc  workclass_Self-emp-not-inc  \
0                  0                       0                           0   
1                 

In [ ]:
print( data["feature_names"], data["target_names"] )
for ff in data["feature_names"]:
  pd.get_dummies(data["data"])

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country'] ['class']


In [ ]:
pd.get_dummies(data["target"]).head()

,<=50K,>50K
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [ ]:
def set_random_seed( seed ):
  random.seed( seed )
  np.random.seed( seed )

set_random_seed( 0 )

# モデルの初期化、クロスバリデーションによる学習と評価

In [ ]:
# 機械学習モデルの初期化
clf = RandomForestClassifier(
    max_depth=8,  # ハイパーパラメータ
    min_samples_split=0.5,  # ハイパーパラメータ
)

In [ ]:
# 交差検証による評価
score = cross_val_score(clf, X, y, cv=3)
accuracy = score.mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.7607182349443268


# Optunaを使って、100回のブラックボックス最適化

In [ ]:
def objective(trial):
    set_random_seed( 0 )

    clf = RandomForestClassifier(
        max_depth=trial.suggest_int(
            "max_depth", 2, 32,
        ),
        min_samples_split=trial.suggest_float(
            "min_samples_split", 0, 1,
        ),
    )

    score = cross_val_score(clf, X, y, cv=3)
    accuracy = score.mean()
    return accuracy

## list_2_14_optimize_rf.py

In [ ]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=100)

print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2024-03-19 15:39:07,592] A new study created in memory with name: no-name-bf81351c-ea09-4557-9b2e-8b8a34444c7a
[I 2024-03-19 15:39:09,480] Trial 0 finished with value: 0.7607182349443268 and parameters: {'max_depth': 19, 'min_samples_split': 0.7151893663724195}. Best is trial 0 with value: 0.7607182349443268.
[I 2024-03-19 15:39:12,507] Trial 1 finished with value: 0.7607182349443268 and parameters: {'max_depth': 20, 'min_samples_split': 0.5448831829968969}. Best is trial 0 with value: 0.7607182349443268.
[I 2024-03-19 15:39:14,251] Trial 2 finished with value: 0.7607182349443268 and parameters: {'max_depth': 15, 'min_samples_split': 0.6458941130666561}. Best is trial 0 with value: 0.7607182349443268.
[I 2024-03-19 15:39:16,797] Trial 3 finished with value: 0.7607182349443268 and parameters: {'max_depth': 15, 'min_samples_split': 0.8917730007820798}. Best is trial 0 with value: 0.7607182349443268.
[I 2024-03-19 15:39:20,703] Trial 4 finished with value: 0.7693788064158434 and parame

Best objective value: 0.85592318661694
Best parameter: {'max_depth': 27, 'min_samples_split': 0.0010546408626361447}


## list_2_14_optimize_rf.py＋storage使用

In [ ]:
study = optuna.create_study(study_name="ch2-rf-seed", storage="sqlite:///optuna.db", direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=100)

print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2024-03-20 06:22:48,549] Using an existing study with name 'ch2-rf-seed' instead of creating a new one.
[I 2024-03-20 06:22:58,802] Trial 100 finished with value: 0.8464231581699796 and parameters: {'max_depth': 14, 'min_samples_split': 0.021362272594642306}. Best is trial 72 with value: 0.8554523013892831.
[I 2024-03-20 06:23:10,816] Trial 101 finished with value: 0.8517054858265473 and parameters: {'max_depth': 18, 'min_samples_split': 0.009011066100582318}. Best is trial 72 with value: 0.8554523013892831.
[I 2024-03-20 06:23:19,036] Trial 102 finished with value: 0.8403218397552283 and parameters: {'max_depth': 19, 'min_samples_split': 0.04818164005763597}. Best is trial 72 with value: 0.8554523013892831.
[I 2024-03-20 06:23:29,129] Trial 103 finished with value: 0.8475492541136544 and parameters: {'max_depth': 20, 'min_samples_split': 0.02326090335375395}. Best is trial 72 with value: 0.8554523013892831.
[I 2024-03-20 06:23:42,828] Trial 104 finished with value: 0.85434670939093

Best objective value: 0.8559231929049508
Best parameter: {'max_depth': 24, 'min_samples_split': 0.0006325520315008212}


## list_2_19_load_study.py

In [ ]:
study = optuna.load_study(
    storage="sqlite:///optuna.db",
    study_name="ch2-rf-seed",
)

print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Best number: {study.best_trial}")
print(f"Best number: {study.best_trial.number}")

Best objective value: 0.8559231929049508
Best parameter: {'max_depth': 24, 'min_samples_split': 0.0006325520315008212}
Best number: FrozenTrial(number=178, state=TrialState.COMPLETE, values=[0.8559231929049508], datetime_start=datetime.datetime(2024, 3, 20, 6, 36, 4, 44871), datetime_complete=datetime.datetime(2024, 3, 20, 6, 36, 20, 969838), params={'max_depth': 24, 'min_samples_split': 0.0006325520315008212}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_split': FloatDistribution(high=1.0, log=False, low=0.0, step=None)}, trial_id=389, value=None)
Best number: 178


In [ ]:
study_df = study.trials_dataframe()
pd.set_option( 'display.max_rows', 20 )
study_df.sort_values( "value", ascending=False )

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_min_samples_split,state
178,178,0.855923,2024-03-20 06:36:04.044871,2024-03-20 06:36:20.969838,0 days 00:00:16.924967,24,0.000633,COMPLETE
72,72,0.855452,2024-03-20 06:10:17.568377,2024-03-20 06:10:33.182839,0 days 00:00:15.614462,23,0.000906,COMPLETE
186,186,0.855391,2024-03-20 06:37:41.889429,2024-03-20 06:37:57.927619,0 days 00:00:16.038190,23,0.000490,COMPLETE
33,33,0.855370,2024-03-20 06:04:55.200855,2024-03-20 06:05:09.940429,0 days 00:00:14.739574,25,0.002708,COMPLETE
197,197,0.855370,2024-03-20 06:39:51.535582,2024-03-20 06:40:08.063843,0 days 00:00:16.528261,25,0.000783,COMPLETE
...,...,...,...,...,...,...,...,...
27,27,0.760718,2024-03-20 06:04:18.422362,2024-03-20 06:04:21.564043,0 days 00:00:03.141681,20,0.460450,COMPLETE
28,28,0.760718,2024-03-20 06:04:21.630460,2024-03-20 06:04:23.553899,0 days 00:00:01.923439,15,0.733025,COMPLETE
29,29,0.760718,2024-03-20 06:04:23.616091,2024-03-20 06:04:26.075505,0 days 00:00:02.459414,23,0.968106,COMPLETE
39,39,0.760718,2024-03-20 06:05:37.691836,2024-03-20 06:05:40.388435,0 days 00:00:02.696599,19,0.552886,COMPLETE


## load_studyを使って、ベストパラメータで学習する

In [ ]:
study.trials[study.best_trial.number]

FrozenTrial(number=178, state=TrialState.COMPLETE, values=[0.8559231929049508], datetime_start=datetime.datetime(2024, 3, 20, 6, 36, 4, 44871), datetime_complete=datetime.datetime(2024, 3, 20, 6, 36, 20, 969838), params={'max_depth': 24, 'min_samples_split': 0.0006325520315008212}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_split': FloatDistribution(high=1.0, log=False, low=0.0, step=None)}, trial_id=389, value=None)

In [ ]:
# ベストトライアルを参照する
tr = study.trials[study.best_trial.number]

# 乱数シードを設定
set_random_seed( 0 )

# 機械学習モデルの初期化
clf = RandomForestClassifier(
    max_depth=tr.params['max_depth'],
    min_samples_split=tr.params['min_samples_split'],
)

# 交差検証による評価
score = cross_val_score(clf, X, y, cv=3)
accuracy = score.mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.8559231929049508


# Optuna Dashboard

In [4]:
!pip install optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 11.4 MB/s eta 0:00:00


## Optuna Dashboardの公式サイトのOptuna Dashboardの起動方法
https://optuna-dashboard.readthedocs.io/en/latest/getting-started.html#google-colaboratory

In [6]:
import optuna
import threading
from google.colab import output
from optuna_dashboard import run_server

# Start Optuna Dashboard
port = 8081
storage = optuna.storages.RDBStorage("sqlite:///optuna.db")
thread = threading.Thread(target=run_server, args=(storage,), kwargs={"port": port})
thread.start()
output.serve_kernel_port_as_window(port, path='/dashboard/')

<IPython.core.display.Javascript object>

## stack overflowのOptuna Dashboardの起動方法
https://stackoverflow.com/questions/76033104/launching-optuna-dashboard-in-google-colaboratory

In [ ]:
import time
import threading
from optuna_dashboard import wsgi
from wsgiref.simple_server import make_server

# Start Optuna Dashboard
port = 8081
storage = optuna.storages.RDBStorage("sqlite:///optuna.db")
app = wsgi(storage)
httpd = make_server("localhost", port, app)
thread = threading.Thread(target=httpd.serve_forever)
thread.start()
time.sleep(3) # Wait until the server startup

from google.colab import output
output.serve_kernel_port_as_window(port, path='/dashboard/')

<IPython.core.display.Javascript object>

# データベースを直接確認する

## JupySQLのインストール
* ipython-sqlは不具合が多く、もうメンテナンスもされてないので使わない
* 代わりに、JupySQLを使用する（使い方はipython-sqlとほぼ同じ）

In [ ]:
!pip install jupysql duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.7/95.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.0/193.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00


## データベースに接続する
* SQLエクステンションを読み込む
* データベースに接続する

In [ ]:
%load_ext sql
%sql sqlite:///optuna.db

Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Connecting to 'sqlite:///optuna.db'

* 表示数を10では少ないので、30に変更する

## Optunaのデータベースの内容を確認する

In [ ]:
%config SqlMagic.displaylimit = 30

In [ ]:
%sql select name from sqlite_master where type='table';

Running query in 'sqlite:///optuna.db'

name
studies
version_info
study_directions
study_user_attributes
study_system_attributes
trials
trial_user_attributes
trial_system_attributes
trial_params
trial_values


In [ ]:
%sql select * from studies;

Running query in 'sqlite:///optuna.db'

study_id,study_name
1,ch2-conditional
2,ch2


In [ ]:
%sql select * from trials where study_id=1;

Running query in 'sqlite:///optuna.db'

trial_id,number,study_id,state,datetime_start,datetime_complete


## sqlite3をインストールして使ってみる
* 使えないことはないが、SQLコマンドの入力がパスワードみたいに***で表示されて使いにくい

In [ ]:
!apt install sqlite3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.3 [768 kB]
Fetched 768 kB in 1s (800 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.3_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.3) ...
Setting up sqlite3 (3.37.2-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!sqlite3 optuna.db

SQLite version 3.37.2 2022-01-06 13:25:41
Enter ".help" for usage hints.
sqlite> .tables
alembic_version            trial_params             
studies                    trial_system_attributes  
study_directions           trial_user_attributes    
study_system_attributes    trial_values             
study_user_attributes      trials                   
trial_heartbeats           version_info             
trial_intermediate_values
sqlite> select * from studies;
1|ch2-conditional
2|ch2
sqlite> select * from trials;
1|0|2|COMPLETE|2024-03-17 10:06:22.116445|2024-03-17 10:06:24.732545
2|1|2|COMPLETE|2024-03-17 10:06:24.780681|2024-03-17 10:06:26.510629
3|2|2|COMPLETE|2024-03-17 10:06:26.550074|2024-03-17 10:06:28.876243
4|3|2|COMPLETE|2024-03-17 10:06:28.940149|2024-03-17 10:06:30.221310
5|4|2|COMPLETE|2024-03-17 10:06:30.268756|2024-03-17 10:06:31.538427
6|5|2|COMPLETE|2024-03-17 10:06:31.575799|2024-03-17 10:06:33.070238
7|6|2|COMPLETE|2024-03-17 10:06:33.114763|2024-03-17 10:06:34.617784